In [ ]:
import os, math, sys
import pandas as pd
import pandapower as pp
import numpy as np

from pandapower.file_io import from_json, to_json

main_dir = os.path.abspath(os.path.join(os.path.abspath(""), ".."))
data_dir = os.path.join(main_dir, 'Modified_116_LV_CSV')

if main_dir not in sys.path:
    sys.path.insert(0, main_dir)

In [ ]:
net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
net48

In [ ]:
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
loadShape_df = pd.read_csv(os.path.join(data_dir, 'LoadShapes.csv'), skiprows=1, sep=';')
loads_df.head(), loadShape_df.head()

merged_load = loads_df.merge(loadShape_df, left_on="Yearly", right_on="Name", how="left")
merged_load.head(10)

In [ ]:
filtered_loads = merged_load[merged_load["Name_x"].isin(net48.asymmetric_load.name.values)]

ts_data = {}
for _, row in filtered_loads.iterrows():
    # print(row["File"])
    file_path = os.path.join(data_dir, 'Load Profiles', row["File"])
    try:
        profile = pd.read_csv(file_path)
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()
    except:
        profile = pd.read_csv(file_path, sep=';')
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()

ts_data

In [ ]:
profile_df = pd.DataFrame(ts_data)
profile_df.head(10)

from pandapower.timeseries.data_sources.frame_data import DFData

ds = DFData(profile_df)

import pandapower.control as ppc

for load_name in net48.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net48.asymmetric_load[net48.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = filtered_loads[filtered_loads["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net48, element_index=idx,
        element="asymmetric_load",
        element_name=f"CTRL_{net48.asymmetric_load.name.at[idx]}",
        variable=f"p_{phase}_mw",
        profile_name=load_name, data_source=ds
    )
net48

In [ ]:
output_dir = os.path.join(main_dir, "timeseries_results")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
from pandapower.timeseries import run_timeseries
from pandapower.timeseries import OutputWriter

net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
net48

# Create and attach output writer to net48
# ow48 = OutputWriter(net48, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# # Log desired variables
# ow48.log_variable('res_trafo_3ph', 'loading_percent', index=net48.trafo.index[0])

# ow48.log_variable('res_bus_3ph', 'vm_b_pu', index=[32, 26]) # LOAD2 and LOAD23
# ow48.log_variable('res_bus_3ph', 'vm_a_pu', index=[14]) # LOAD55

# run_timeseries(net48, time_steps=range(profile_df.shape[0]), run=pp.runpp_3ph, continue_on_divergence=False)

In [ ]:
net = from_json(os.path.join(main_dir, 'json_networks', 'no_load_network.json'))

# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in loads_df.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in net.bus.name) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=net.bus.index[net.bus.name==bus_id][0],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']
net

In [ ]:
ts_data = {}
for _, row in merged_load.iterrows():
    # print(row["File"])
    file_path = os.path.join(data_dir, 'Load Profiles', row["File"])
    try:
        profile = pd.read_csv(file_path)
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()
    except:
        profile = pd.read_csv(file_path, sep=';')
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()

len(ts_data)
profile_df = pd.DataFrame(ts_data)
print(profile_df.shape)
profile_df.head(10)

In [ ]:
from pandapower.timeseries.data_sources.frame_data import DFData

ds = DFData(profile_df)

import pandapower.control as ppc

for load_name in net.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net.asymmetric_load[net.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = merged_load[merged_load["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net, element_index=idx,
        element="asymmetric_load",
        element_name=f"CTRL_{net.asymmetric_load.name.at[idx]}",
        variable=f"p_{phase}_mw",
        profile_name=load_name, data_source=ds
    )
net

In [ ]:
# Create and attach output writer to net48
ow = OutputWriter(net, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# Log desired variables
ow.log_variable('res_trafo_3ph', 'loading_percent', index=net.trafo.index[0])

ow.log_variable('res_bus_3ph', 'vm_a_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=net.bus.index)

run_timeseries(net48, time_steps=range(profile_df.shape[0]), continue_on_divergence=False)

In [ ]:
vm_a_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph\\vm_a_pu.p'))
vm_b_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph\\vm_b_pu.p'))
vm_c_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph\\vm_c_pu.p'))

vm_a_pu